In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "<elpatron1m"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = "GraziosoSalvareLogo.png"
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),

    html.Div([
        # Adding unique identifier
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}),
        html.Center(html.H3('Created by Yakuba Conde'))
    ]),

    html.Hr(),

    html.Div(
        # FIX ME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-options',  # FIXED: Changed from 'filter-type' to match input ID
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset (Show All)', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ),

    html.Hr(),

    dash_table.DataTable(
        page_size=10,
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # FIX ME: Set up the features for your interactive data table to make it user-friendly for your client
    ),

    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-options', 'value')])  # FIXED: Changed from 'filter-type' to 'filter-options'
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries

    if filter_type == "Reset":
        filtered_df = df  # Show all data when reset is selected
    else:
        filtered_df = df[df['breed'].str.contains(filter_type, case=False, na=False)]  # Filter based on breed

    return filtered_df.to_dict('records')  # Returning only data, as columns are static


# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ### FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    if viewData is None:
        filtered_df = df  # Use original data if no filters are applied
    else:
        filtered_df = pd.DataFrame(viewData)  # Convert viewData to a DataFrame

    return [
        dcc.Graph(
            figure=px.pie(filtered_df, names='breed', title='Preferred Animals')
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

if __name__ == '__main__':  
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:10595/
